In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5fe53b184f367847aa55a2617e1b05248cac098b2c3dd14dae217169b3ce2e42b10bcc2af6d9b6787a4792a557f9705bb1eba86d761f2407d2'

In [2]:
import aocd
from aocd.models import Puzzle
day = 7
year = 2022
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [43]:
test_data = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

In [87]:
data = puzzle.input_data.splitlines()
# data = test_data.splitlines()
len(data), data[:10]

(1014,
 ['$ cd /',
  '$ ls',
  'dir blgtdv',
  'dir dbrfcz',
  'dir fvspj',
  'dir hbjmndt',
  'dir hzg',
  'dir jpjgdm',
  'dir mtd',
  'dir pcpf'])

In [88]:
import re
import pandas as pd
max_size = 100000

In [89]:
from collections import defaultdict, deque

class FileSystem():
    def __init__(self):
        self.rec_dd = lambda: defaultdict(self.rec_dd)
        self.tree = defaultdict(self.rec_dd)
    
    def traverse(self, dirs):
        dirs = deque(dirs)
        cd = self.tree
        while len(dirs):
            d = dirs.popleft()
            cd = cd[d]
        return cd
    
    def add_file(self, dirs, file, size):        
        cd = self.traverse(dirs)
        cd[file] = size
    
    def touch(self, dirs):
        cd = self.traverse(dirs)
        cd = defaultdict(self.rec_dd)
    
    @staticmethod
    def dir_sizes(d):
        total = 0
        sub_dirs = []
        for key in d:
            if isinstance(d[key], int):
                # print('adding', key, d[key])
                total += d[key]
            else:
                sub_dirsize, directories = FileSystem.dir_sizes(d[key])
                total += sub_dirsize
                sub_dirs += directories
                sub_dirs += [(key, sub_dirsize)]
        return total, sub_dirs
    
    def get_dir_sizes(self):
        return self.dir_sizes(self.tree)
        

In [90]:
def parse(stdout):
    fs = FileSystem()
    dir_stack = []
    for line in stdout:
        if line[0] == '$':
            if line == '$ ls':
                pass
            elif line[5:] == '/':
                dir_stack = []
            elif line[5:] == '..':
                dir_stack.pop()
            else:
                dir_stack.append(line[5:])
        else:
            if line.startswith('dir'):
                fs.touch(dir_stack + line.split(' ')[1:])
            else:
                size, file = line.split(' ')
                fs.add_file(dir_stack, file, int(size))
    return fs

In [91]:
fs = parse(data)

In [95]:
used_space, subdirs = fs.get_dir_sizes()
df = pd.DataFrame(subdirs, columns=['dirname', 'dirsize'])
result = df[(df.dirsize <= max_size)].dirsize.sum()
result

1667443

In [93]:
puzzle.answer_a = result

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


# Part 2

In [96]:
total_space = 70000000
needed_space = 30000000
free_space = total_space - used_space

In [97]:
free_space

22129546

In [108]:
resultB = df[(df.dirsize + free_space >= needed_space)].sort_values('dirsize').dirsize.iloc[0]
resultB

8998590

In [109]:
puzzle.answer_b = resultB

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 7! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [110]:
df

,dirname,dirsize
0,fvspj,327970
1,scggvb,29012
2,bjp,29012
3,drsgv,549918
4,lpv,596985
...,...,...
177,mwzt,302445
178,gqpfcv,302445
179,lpv,38004
180,qtgqnqh,175846
